In [1]:

from importlib import reload

import h5py
import numpy as np

import utilities

reload(utilities)

<module 'utilities' from "/Users/gloirelinvani/Library/Mobile Documents/com~apple~CloudDocs/School/Magistère d'Informatique/LDD3/S6/Stage/Earthquakes/earthquakes/New_code/utilities.py">

# Functions to build displacement maps and labels for each main shock in input HDF5 file

In [2]:
## Global variables:
n_seq_init = 0  # Number of sequences processed
n_seq_discarded = 0  # Number of sequences discarded


def process_main_shock_data(hdf5_file_path, output_path):
    """Process main shocks data to generate interpolated displacement maps."""
    with h5py.File(hdf5_file_path, 'r') as file:
        for id_seq in file.keys():
            process_single_main_shock(file[id_seq], output_path, id_seq)


def process_single_main_shock(main_shock_group, output_path, id_seq):
    global n_seq_init, n_seq_discarded
    n_seq_init += 1
    print(f"Processing main shock {id_seq}...")
    gps_stations_displacements = main_shock_group['gps_stations_displacements'][()]
    # For each station, multiply the displacement by the scale factor and sum the displacements
    # over the time dimension
    ## gps_stations_displacements: (NbStations, NbDays, Displacement(X,Y,Z))
    gps_stations_displacements *= scale_factor  ## convert to mm
    gps_stations_displacements = gps_stations_displacements.reshape(-1, 3)  ## (NbStations, (X,Y,Z))
    gps_stations_positions = main_shock_group['stations_positions'][()]
    main_shock_location = main_shock_group.attrs['main_shock_location']
    after_shocks_locations = main_shock_group['aftershocks_locations'][()]
    after_shocks_locations = after_shocks_locations.reshape(-1, 2)  ## (NbAfterShocks, (X,Y))
    main_shock_day = main_shock_group.attrs['main_shock_day']
    main_shock_mag = main_shock_group.attrs['main_shock_magnitude']

    # Calculate the interpolation grid
    min_lat, max_lat, min_lon, max_lon, n_pixels_lat, n_pixels_lon = utilities.calculate_interpolation_grid(
        main_shock_location, cell_size_degs, int(num_cells / 2)
    )

    # Number of stations inside the grid
    # n_stations_inside_grid = ((gps_stations_positions[:, 0] >= min_lat) & (gps_stations_positions[:, 0] <= max_lat) &
    #                           (gps_stations_positions[:, 1] >= min_lon) & (
    #                                       gps_stations_positions[:, 1] <= max_lon)).sum()
    # 
    # # Skip if not enough stations
    # if n_stations_inside_grid < min_stations_inside:
    #     n_seq_discarded += 1
    #     print('Skipping:', id_seq, 'with only', n_stations_inside_grid,
    #           'stations inside the grid, out of total (downloaded) ',
    #           gps_stations_positions.shape[0])
    #     return

    # labels
    labels = np.zeros((n_pixels_lat, n_pixels_lon))
    # discretize the aftershocks 
    aftershocks_rows = ((after_shocks_locations[:, 0] - min_lat) / cell_size_degs).astype('int')
    aftershocks_cols = ((after_shocks_locations[:, 1] - min_lon) / cell_size_degs).astype('int')
    # mask to make sure no event is outside the grid
    aftershocks_mask = (aftershocks_rows < n_pixels_lat) & (aftershocks_rows >= 0) & (
            aftershocks_cols < n_pixels_lon) & (
                               aftershocks_cols >= 0)
    aftershocks_rows = aftershocks_rows[aftershocks_mask]
    aftershocks_cols = aftershocks_cols[aftershocks_mask]
    # Metadata on aftershocks inside the grid for the plots
    grid_after_shocks_locations = after_shocks_locations[aftershocks_mask]
    grid_after_shocks_magnitudes = main_shock_group['aftershocks_magnitudes'][()][aftershocks_mask]
    if aftershocks_mask.sum() == 0:
        n_seq_discarded += 1
        print('Skipping:', id_seq, 'with no aftershocks inside the grid')
        return

    if not regression:
        if soft_labels:
            utilities.create_classification_soft_labels(labels, aftershocks_rows, aftershocks_cols, cell_size_rads,
                                                        sigma_softlabels * cell_size_rads)
        else:
            labels[aftershocks_rows, aftershocks_cols] = 1
    else:
        seismic_moments = 10 ** (
                1.5 * grid_after_shocks_magnitudes + 6.07)  # aftershocks magnitudes converted to their seismic moments
        if soft_labels:
            utilities.create_regression_soft_labels(labels, seismic_moments,
                                                    aftershocks_rows, aftershocks_cols, cell_size_rads,
                                                    sigma_softlabels * cell_size_rads)
        else:
            #print(aftershocks_rows)
            #print(aftershocks_cols)
            np.add.at(labels, (aftershocks_rows, aftershocks_cols),
                      seismic_moments)  # add the seismic moments to the grid (sum of seismic moments in case of multiple aftershocks in the same pixel)
            #print("Seismic moments", seismic_moments)
            #print(grid_after_shocks_magnitudes)
            #print(labels[aftershocks_rows, aftershocks_cols])
            #sys.exit()

    # The grid of interpolated displacements: channel shape (X,Y,Z, distance pixel-MS, flag to indicate if the pixel is valid or not)
    interpolated_displacements = np.zeros((n_pixels_lat, n_pixels_lon,
                                           5))
    if elasticity:
        try:
            gps_stations_displacements_means = gps_stations_displacements.mean(axis=0)
            gps_stations_displacements_stds = gps_stations_displacements.std(axis=0)
            gps_stations_displacements_standardized = (
                                                              gps_stations_displacements - gps_stations_displacements_means) / gps_stations_displacements_stds
            # gps_stations_displacements_standardized = zscore(gps_stations_displacements, axis=0) # Standardize the displacements
            automatic_reg_factor = 2  #minimal_distances[id]/cell_size_km

            utilities.elasticity_interpolation(
                interpolated_displacements,
                np.pi * main_shock_location / 180,
                np.pi * gps_stations_positions / 180,
                gps_stations_displacements_standardized,
                n_pixels_lat, n_pixels_lon,
                np.pi * min_lat / 180,
                np.pi * min_lon / 180,
                cell_size_rads,
                sigma_rads,
                min_station_pixel_distance_km,
                reg_factor=automatic_reg_factor,
                index_ratio=0.8)

            interpolated_displacements[:, :, :2] *= gps_stations_displacements_stds[:2]
            interpolated_displacements[:, :, :2] += gps_stations_displacements_means[:2]
        except Exception as e:
            raise e

    else:
        utilities.interpolate_displacements(
            interpolated_displacements,
            np.pi * main_shock_location / 180,
            np.pi * gps_stations_positions / 180,
            gps_stations_displacements,
            n_pixels_lat, n_pixels_lon,
            np.pi * min_lat / 180,
            np.pi * min_lon / 180,
            cell_size_rads,
            sigma_rads,
            min_station_pixel_distance_km,
        )
    #Save the interpolated data and labels to the output file
    save_interpolated_data(interpolated_displacements, labels, output_path, id_seq, main_shock_day, main_shock_mag,
                           main_shock_location, grid_after_shocks_locations, grid_after_shocks_magnitudes,
                           gps_stations_positions)


def save_interpolated_data(interpolated_data, labels, output_path, main_shock_id, main_shock_day, main_shock_mag,
                           main_shock_location, grid_after_shocks_locations, grid_after_shocks_magnitudes,
                           gps_stations_positions):
    with h5py.File(output_path, 'a') as f:
        if str(main_shock_id) in f:
            del f[str(main_shock_id)]
        grp = f.create_group(str(main_shock_id))
        grp.attrs['main_shock_day'] = main_shock_day
        grp.attrs['main_shock_magnitude'] = main_shock_mag
        # For the plots
        grp.attrs['main_shock_location'] = main_shock_location

        grp.create_dataset('interpolated_displacement', data=interpolated_data)
        grp.create_dataset('labels', data=labels)
        # Metadata for the plots
        grp.create_dataset('grid_after_shocks_locations', data=grid_after_shocks_locations)
        grp.create_dataset('grid_after_shocks_magnitudes', data=grid_after_shocks_magnitudes)
        grp.create_dataset('gps_station_positions', data=gps_stations_positions)

# Parameters

In [5]:
earth_radius_km = 6371  # km
grid_size_km = 250  # Size of the square grid in km
cell_size_km = 5  #Cell size in km
cell_size_rads = cell_size_km / earth_radius_km  # Cell size in radians
cell_size_degs = cell_size_rads * 180 / np.pi
scale_factor = 1e6  # Scale factor for the coordinates from km to mm
num_cells = int(grid_size_km / cell_size_km)  # number of cells in the lateral direction
#min_stations_inside = 3  # minimal number of stations inside the grid
soft_labels = False  ## smoothed labels (smoothed over space)
regression = False  ## regression instead of classification
elasticity = True  # Thin 2D elastic sheet model for interpolating GPS data
sigma_softlabels = 1  # sigma for the gaussian smoothing of the labels, expressed in number of cell sizes
sigma_interpolation = 8  ## expressed in number of cell sizes
sigma_rads = sigma_interpolation * cell_size_rads  # sigma for the gaussian interpolation
min_station_pixel_distance_km = 86  # minimal distance between station and pixel in km
hdf5_in_put_file_path = "Data/Displacements_min_mainshock_mag=6_min_stations_per_main_shock=3_regression=True_min_after_shock_mag=2.5_after_shock_time_window=45_n_days_before_mainshock=1_n_days_after_mainshock=1_min_days_between_mainshocks=30_grid_size_km=250.hdf5"  #Path to the HDF5 output file
hdf5_out_put_file_path = (
    f"Data/Interpolated_Data_reg={regression}_soft_labels={soft_labels}_elasticity={elasticity}_min_mainshock_mag=6_min_stations_per_main_shock=3_min_after_shock_mag=4_after_shock_time_window=45_n_days_before_mainshock=1_n_days_after_mainshock=1.hdf5")  #Path to the HDF5 output file

# Main

In [ ]:
def main():
    process_main_shock_data(hdf5_in_put_file_path, hdf5_out_put_file_path)


if __name__ == '__main__':
    main()
    print('Number of sequences processed:', n_seq_init)
    print('Number of sequences discarded:', n_seq_discarded)
    print('Number of sequences kept:', n_seq_init - n_seq_discarded)

Processing main shock 103...
Processing main shock 109...
Processing main shock 114...
Processing main shock 118...
Processing main shock 151...
Processing main shock 155...
Processing main shock 156...
Processing main shock 158...
Processing main shock 161...
Processing main shock 163...
Processing main shock 172...
Processing main shock 174...
Processing main shock 176...
Processing main shock 179...
Processing main shock 181...
Processing main shock 182...
Processing main shock 187...
Processing main shock 194...
Processing main shock 198...
Processing main shock 199...
Processing main shock 200...
Processing main shock 203...
Processing main shock 204...
Processing main shock 206...
Processing main shock 207...
Processing main shock 213...
Processing main shock 214...
Processing main shock 215...
Processing main shock 217...
Processing main shock 218...
Processing main shock 219...
Processing main shock 226...
Processing main shock 233...
Processing main shock 248...
Processing mai